In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import ast
from itertools import combinations

In [45]:
df = pd.read_csv('cleaned_ages_dataset.csv')
df.head()

,Name,Gender,Occupation,Birth year,Death year,Manner of death,Age of death,Associated Countries,Associated Country Life Expectancy,AOD_Category,Gender Grouped
0,douglas adams,['male'],artist,1952,2001,natural causes,49,['united kingdom'],[81.3],41-50,male
1,abraham lincoln,['male'],politician,1809,1865,homicide,56,['united states'],[78.5],51-60,male
2,paul morand,['male'],artist,1888,1976,NaN,88,['france'],[82.5],81-90,male
3,claude monet,['male'],artist,1840,1926,natural causes,86,['france'],[82.5],81-90,male
4,elvis presley,['male'],artist,1935,1977,natural causes,42,['united states'],[78.5],41-50,male


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742532 entries, 0 to 742531
Data columns (total 11 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Name                                742532 non-null  object
 1   Gender                              666992 non-null  object
 2   Occupation                          684916 non-null  object
 3   Birth year                          742532 non-null  int64 
 4   Death year                          742532 non-null  int64 
 5   Manner of death                     43779 non-null   object
 6   Age of death                        742532 non-null  int64 
 7   Associated Countries                742532 non-null  object
 8   Associated Country Life Expectancy  742304 non-null  object
 9   AOD_Category                        742532 non-null  object
 10  Gender Grouped                      742532 non-null  object
dtypes: int64(3), object(8)
memory usage: 62

### Data Preprocessing

The data must be converted into a format that can be used for the Apriori algorithm. This means converting each item in the dataset into a 'transaction'. Each transaction is a set of attribute values, we use six columns for this: 'Gender', 'Occupation', 'Manner of death', 'Age of death', 'AOD_Category', and 'Associated Countries'.

an example of a transaction:<br>
{’AOD_Category=41-50’,’Age of death=49’,’Associated Countries=united kingdom’,’Gender=male’,’Manner of death=natural causes’,’Occupation=artist’}


In [47]:
# Remove rows with null values in 'Associated Countries' and 'Gender' columns
df_with_mod = df.dropna(subset=['Associated Countries', 'Gender'])
df_without_mod = df_with_mod.copy()
df_without_mod.drop('Manner of death', axis=1, inplace=True)
selected_columns = ['Gender', 'Occupation', 'Age of death', 'AOD_Category', 'Associated Countries']

# Initialize an empty list to hold the transactions
transactions = []

# Convert each row in the dataset to a transaction
for index, row in df_without_mod.iterrows():
    transaction = []
    for col in selected_columns:
        # Special handling for 'Associated Countries' and 'Gender' columns which may have multiple values
        if col == 'Associated Countries' or col == 'Gender':
            # Convert the string representation to a list
            values = ast.literal_eval(row[col]) if isinstance(row[col], str) else row[col]
            for value in values:
                item = f"{col}={value}"
                transaction.append(item)
        else:
            # Create an item by combining the column name and the value (e.g., "Gender=male")
            item = f"{col}={row[col]}"
            transaction.append(item)
    transactions.append(set(transaction))

# Show the first 5 final revised transactions
transactions[:5]


[{'AOD_Category=41-50',
  'Age of death=49',
  'Associated Countries=united kingdom',
  'Gender=male',
  'Occupation=artist'},
 {'AOD_Category=51-60',
  'Age of death=56',
  'Associated Countries=united states',
  'Gender=male',
  'Occupation=politician'},
 {'AOD_Category=81-90',
  'Age of death=88',
  'Associated Countries=france',
  'Gender=male',
  'Occupation=artist'},
 {'AOD_Category=81-90',
  'Age of death=86',
  'Associated Countries=france',
  'Gender=male',
  'Occupation=artist'},
 {'AOD_Category=41-50',
  'Age of death=42',
  'Associated Countries=united states',
  'Gender=male',
  'Occupation=artist'}]

In [48]:
# Sample code to print out transactions with multiple 'Associated Countries' and 'Gender'
multigendertransactions = [transaction for transaction in transactions 
                            if  len([item for item in transaction if 'Gender=' in item]) > 1]

multigendertransactions[:2]


[{'AOD_Category=71-80',
  'Age of death=71',
  'Associated Countries=germany',
  'Associated Countries=israel',
  'Gender=intersex',
  'Gender=transgender male',
  'Gender=transgender person',
  'Occupation=artist'},
 {'AOD_Category=61-70',
  'Age of death=65',
  'Associated Countries=united states',
  'Gender=female',
  'Gender=transgender female',
  'Occupation=artist'}]

### Item Frequency

In [49]:

# Calculate the frequency of each item across all transactions
item_frequency = defaultdict(int)

# Iterate through each transaction and count the occurrence of each item
for transaction in transactions:
    for item in transaction:
        item_frequency[item] += 1

# Sort items by frequency
sorted_items = sorted(item_frequency.items(), key=lambda x: x[1], reverse=True)

# Show some of the most frequent items
sorted_items[:10]


[('Gender=male', 594457),
 ('Occupation=artist', 180230),
 ('AOD_Category=71-80', 179211),
 ('AOD_Category=81-90', 161078),
 ('Associated Countries=united states', 147009),
 ('AOD_Category=61-70', 128614),
 ('Occupation=politician', 121811),
 ('Occupation=athlete', 89928),
 ('Associated Countries=germany', 77554),
 ('Associated Countries=united kingdom', 75757)]

### Generate Candidate Itemsets


In [75]:
def calculate_support_no_optimisations(itemset, transactions):
    itemset_count = 0
    total_transactions = len(transactions)
    for transaction in transactions:
        if itemset.issubset(transaction):
            itemset_count += 1
    support = itemset_count / total_transactions
    return support


def generate_candidates__no_optimisations(prev_frequent_itemsets):
    new_candidates = set()
    for i in prev_frequent_itemsets:
        for j in prev_frequent_itemsets:
            union_set = i.union(j)
            if len(union_set) == len(i) + 1:
                new_candidates.add(union_set)
    return new_candidates


def apriori_no_optimisations(transactions, min_support=0.1):
    frequent_itemsets = []
    
    # Generate initial candidates (1-itemsets)
    initial_candidates = set()
    for transaction in transactions:
        for item in transaction:
            initial_candidates.add(frozenset([item]))
            
    # Find frequent 1-itemsets
    frequent_1_itemsets = set()
    for itemset in initial_candidates:
        if calculate_support_no_optimisations(itemset, transactions) >= min_support:
            frequent_1_itemsets.add(itemset)
    frequent_itemsets.extend(frequent_1_itemsets)
    
    prev_frequent_itemsets = frequent_1_itemsets
    
    # Generate new candidates without pruning
    while len(prev_frequent_itemsets) > 0:
        new_candidates = generate_candidates__no_optimisations(prev_frequent_itemsets)
        new_frequent_itemsets = set()
        for itemset in new_candidates:
            if calculate_support_no_optimisations(itemset, transactions) >= min_support:
                new_frequent_itemsets.add(itemset)
        frequent_itemsets.extend(new_frequent_itemsets)
        prev_frequent_itemsets = new_frequent_itemsets
        
    return frequent_itemsets

min_support = 0.1
min_hash_count = 2  # Minimum count for hash table
subset_transactions = transactions[:10000]  # Adjust the subset size as needed
frequent_itemsets = apriori_no_optimisations(subset_transactions, min_support)
frequent_itemsets[:10]


[frozenset({'Occupation=politician'}),
 frozenset({'Occupation=researcher'}),
 frozenset({'Associated Countries=germany'}),
 frozenset({'Gender=female'}),
 frozenset({'AOD_Category=61-70'}),
 frozenset({'AOD_Category=81-90'}),
 frozenset({'Associated Countries=austria'}),
 frozenset({'AOD_Category=71-80'}),
 frozenset({'Associated Countries=united states'}),
 frozenset({'AOD_Category=51-60'})]

### Optimizations to the Apriori Algorithm

The original Apriori was slow for big datasets. I implemented these four improvements:

1. **Early Pruning**: I cut down the list of possible item sets early on. This means fewer item sets to check later.
   
2. **Transaction Reduction**: I removed data rows that were already fully explained by frequent item sets. This makes the data smaller and faster to scan.

3. **Hash Method**: I used a hash table to keep track of item sets. This is faster than using lists or sets.

4. **Twice-Only Partitioning**: I split the data into smaller parts and ran Apriori on each. The results were then used for a final check on the full data. This means fewer item sets to check in the end.

These changes did not make any noticeable difference to the efficiency of the algorithm. This was baffling until I discovered  using the library apriori functions did not yield any association rules.

#### Why Optimisations Dont Help

**Low Support**: If the support for all multi-item itemsets in the dataset is below the minimum threshold the algorithm will spend a lot of time generating and checking itemsets that end up being discarded, which will render most optomisations ineffective.

**Sparse Data**: If the data is sparse, meaning there are few recurring patterns of items appearing together, then most of the candidate itemsets would be infrequent, leading to a large number of checks but not a lot of pruning or transaction reductions.

**Lack of Redundant Transactions**: If the dataset doesn't have many redundant transactions, then the transaction reduction step would not help much.

**Ineffective Pruning**: If the data is such that very few itemsets can be pruned early, the pruning step becomes almost useless, leading to a full scan of the dataset for each candidate set.

**Highly Varied Data**: If the data has high variance in terms of the items in transactions, then the hash-based technique and transaction reduction would not provide much benefit.

**Lack of Strong Associations**: Finally, the absence of association rules suggests that the data doesn't have strong associations between items. In this case, the algorithm has to go through the complete search space without finding anything substantial, making optimizations less effective.


In [83]:
# Simple hash function using ascii values of the first letter of each item
def simple_hash(itemset):
    return sum(ord(item[0]) for item in itemset)

def calculate_support(itemset, transactions):
    itemset_count = 0
    total_transactions = len(transactions)
    for transaction in transactions:
        if itemset.issubset(transaction):
            itemset_count += 1
    support = itemset_count / total_transactions
    return support


def generate_and_prune_candidates(prev_frequent_itemsets, min_hash_count=2):
    new_candidates = set()
    hash_table = {}
    
    # Generate candidate itemsets
    for i in prev_frequent_itemsets:
        for j in prev_frequent_itemsets:
            union_set = i.union(j)
            if len(union_set) == len(i) + 1:
                new_candidates.add(union_set)
                
                # Apply hash function
                hash_value = simple_hash(union_set)
                
                # Update hash table
                if hash_value in hash_table:
                    hash_table[hash_value] += 1
                else:
                    hash_table[hash_value] = 1
    
    # Prune candidates using hash table
    pruned_candidates = set()
    for itemset in new_candidates:
        hash_value = simple_hash(itemset)
        if hash_table.get(hash_value, 0) >= min_hash_count:
            pruned_candidates.add(itemset)
            
    return pruned_candidates

def remove_redundant_transactions(transactions, frequent_itemsets):
    # Initialize an empty list to hold the new transactions after removal
    new_transactions = []
    
    # Loop through each transaction in the list of transactions
    for transaction in transactions:
        is_redundant = True  # Assume the transaction is redundant until proven otherwise
        
        # Loop through each item in the transaction
        for item in transaction:
            # Check if this item is part of any frequent itemset
            is_item_in_frequent_itemset = False
            for frequent_itemset in frequent_itemsets:
                if frozenset([item]).issubset(frequent_itemset):
                    is_item_in_frequent_itemset = True
                    break
            
            # If the item is not in any frequent itemset, then the transaction is not redundant
            if not is_item_in_frequent_itemset:
                is_redundant = False
                break
        
        # If the transaction is not redundant, add it to the new list of transactions
        if not is_redundant:
            new_transactions.append(transaction)
            
    return new_transactions


def apriori_with_hashing(transactions, min_support=0.1, min_hash_count=2):
    frequent_itemsets = []
    
    # Generate initial candidates (1-itemsets)
    initial_candidates = set()
    for transaction in transactions:
        for item in transaction:
            initial_candidates.add(frozenset([item]))
            
    # Find frequent 1-itemsets
    frequent_1_itemsets = set()
    for itemset in initial_candidates:
        if calculate_support(itemset, transactions) >= min_support:
            frequent_1_itemsets.add(itemset)
    frequent_itemsets.extend(frequent_1_itemsets)
    
    prev_frequent_itemsets = frequent_1_itemsets
    
    # Generate and prune new candidates
    while len(prev_frequent_itemsets) > 0:
        # Remove redundant transactions
        transactions = remove_redundant_transactions(transactions, prev_frequent_itemsets)
        # Generate and prune candidates using hash-based technique
        new_candidates = generate_and_prune_candidates(prev_frequent_itemsets, min_hash_count)
        new_frequent_itemsets = set()
        for itemset in new_candidates:
            if calculate_support(itemset, transactions) >= min_support:
                new_frequent_itemsets.add(itemset)
        frequent_itemsets.extend(new_frequent_itemsets)
        prev_frequent_itemsets = new_frequent_itemsets
        
    return frequent_itemsets


def apriori_with_twice_only_partitioning(transactions, min_support=0.1):
    # Automatically calculate the partition size to evenly divide the dataset
    num_partitions = 10  # You can change this number as needed
    partition_size = len(transactions) // num_partitions
    
    # Create partitions
    partitions = [transactions[i:i + partition_size] for i in range(0, len(transactions), partition_size)]
    
    local_frequent_itemsets = set()
    for partition in partitions:
        local_frequent = apriori_with_hashing(partition, min_support)
        local_frequent_itemsets.update(local_frequent)
    
    global_frequent_itemsets = set()
    for itemset in local_frequent_itemsets:
        if calculate_support(itemset, transactions) >= min_support:
            global_frequent_itemsets.add(itemset)
            
    return global_frequent_itemsets
# Test the Apriori algorithm with Hash-based technique, using a subset of transactions and a minimum support of 0.1
# Using a subset for testing purposes due to computational intensity
min_support_with_hashing = 0.5
min_hash_count = 2  # Minimum count for hash table
subset_transactions_with_hashing = transactions[:100000]  # Adjust the subset size as needed
frequent_itemsets_with_hashing = apriori_with_hashing(subset_transactions_with_hashing, min_support_with_hashing, min_hash_count)

# Show some frequent itemsets
frequent_itemsets_with_hashing[:10]

[frozenset({'Gender=male'})]

### Pruning and Filtering